In [12]:
import numpy as np
import pandas as pd
import json
import re
from collections import Counter
import little_mallet_wrapper
import seaborn
import glob
from pathlib import Path

#LOADING EVERYTHING
#directory = "/Users/joshs/Desktop/Spring 2021/HUM346/wsj-nyt-oped"

In [13]:
#Importing stuff from NYTIMES
from pynytimes import NYTAPI
import urllib3, json, datetime
nyt = NYTAPI("nKBpzKBvOYipTxZf1T1NwCsVeXad5xoE", parse_dates=True)
articles = nyt.article_search(
    results = 50,
    dates = {
        "begin": datetime.datetime(2020, 1, 31),
        "end": datetime.datetime(2020, 12, 30)
    },
    options = {
        "sources": [
            "New York Times",
        ],
        "news_desk": [
            "OpEd",
            "Editorial"
            
        ]
    }
)

In [16]:
#Writing articles to be saved
for article in articles:
    article['abstract'] = re.sub('\?', '', article['abstract'])
    text_file = open(f"./Articles/{article['abstract']}.txt", "w+")
    n = text_file.write(article['lead_paragraph'] + " \n", )
    text_file.close()

In [17]:
#splitting text into all lowercase
def split_into_words(any_chunk_of_text):
    lowercase_text = any_chunk_of_text.lower()
    split_words = re.split("\W+", lowercase_text) 
    return split_words

In [18]:
#Combining all data together IT SHOULD WORK NOW BUT WAS NOT WORKING EARLIER
files = glob.glob("./Articles/*.txt")
training_data = []
for file in files:
    text = open(file).read()
    processed_text = little_mallet_wrapper.process_string(text)
    training_data.append(processed_text)

In [21]:
original_texts = []
for file in files:
    text = open(file, encoding='cp1252').read()
    original_texts.append(text)

In [22]:
full_text = open("./articles.txt", encoding="utf-8").read()
number_of_desired_words = 10
# Manipulate and Analyze File
stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours',
'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers',
 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are',
 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does',
 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into',
 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down',
 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here',
 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',
 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so',
 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', 've', 'll', 'amp']


all_the_words = split_into_words(full_text)
meaningful_words = [word for word in all_the_words if word not in stopwords]
meaningful_words_tally = Counter(meaningful_words)
most_frequent_meaningful_words = meaningful_words_tally.most_common(number_of_desired_words)

In [23]:
training_data

['monday congress authorized creation latino museum part NUM NUM trillion year end spending bill long overdue decades activists fighting establish latino museum washington mall creation essential documenting community contribution education future americans',
 'many black americans say wary taking coronavirus vaccine logical result centuries abuse exploitation united states health care system empathize patient told one baby trust vaccines friends family say going wait',
 'last decades congress diversified important ways gotten less white less male less straight positive developments staring one many recent senate hearings filled usual magisterial blustering self important yada yada dawned way congress moved wrong direction become quite brazenly unrepresentative',
 'routine home angie sinopoli talkative matriarch large italian family heaped praise children grandchildren even memory faded youngest son steven came house cooked dinner nearly every night couple falls bouts rehabilitation ce

In [24]:
path_to_mallet = 'C:/mallet-2.0.8/bin/mallet'
num_topics = 5
    
#Change to your desired output directory
output_directory_path = './topic-model-output'

#No need to change anything below here
Path(f"{output_directory_path}").mkdir(parents=True, exist_ok=True)

path_to_training_data           = f"{output_directory_path}/training.txt"
path_to_formatted_training_data = f"{output_directory_path}/mallet.training"
path_to_model                   = f"{output_directory_path}/mallet.model.{str(num_topics)}"
path_to_topic_keys              = f"{output_directory_path}/mallet.topic_keys.{str(num_topics)}"
path_to_topic_distributions     = f"{output_directory_path}/{str(num_topics)}"


In [26]:
#Importing Data
little_mallet_wrapper.import_data(path_to_mallet,
                path_to_training_data,
                path_to_formatted_training_data,
                training_data)

Importing data...
Complete


In [27]:
#Training Data Doesnt seem to work?
little_mallet_wrapper.train_topic_model(path_to_mallet,
                      path_to_formatted_training_data,
                      path_to_model,
                      path_to_topic_keys,
                      path_to_topic_distributions,
                      num_topics)

Training topic model...
Complete


In [28]:
topics = little_mallet_wrapper.load_topic_keys(path_to_topic_keys)

for topic_number, topic in enumerate(topics):
    print(f"✨Topic {topic_number}✨\n\n{topic}\n")

✨Topic 0✨

['one', 'world', 'house', 'even', 'america', 'every', 'like', 'public', 'seemed', 'trump', 'election', 'mother', 'economic', 'might', 'would', 'used', 'country', 'less', 'important', 'last']

✨Topic 1✨

['NUM', 'many', 'new', 'months', 'coronavirus', 'states', 'home', 'care', 'live', 'restaurants', 'night', 'united', 'spending', 'london', 'government', 'percent', 'workers', 'unemployed', 'millions', 'covid']

✨Topic 2✨

['pandemic', 'life', 'small', 'across', 'always', 'businesses', 'menopause', 'economy', 'becomes', 'come', 'political', 'ago', 'help', 'aid', 'things', 'personal', 'large', 'often', 'industry', 'fashion']

✨Topic 3✨

['time', 'years', 'two', 'congress', 'could', 'foreign', 'court', 'right', 'men', 'much', 'recently', 'came', 'health', 'discussed', 'abreu', 'nolan', 'ind', 'haute', 'terre', 'red']

✨Topic 4✨

['year', 'get', 'children', 'say', 'americans', 'long', 'power', 'around', 'means', 'women', 'knew', 'back', 'president', 'payments', 'scale', 'business'